<center><h1 style="background-color:#8ffaaa;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 10px;padding: 5px"><b><u>Project: Which customer will respond to direct mail?</u></b></h1></center>
<center><h2><b>Springleaf Marketing Response</b></h2></center>
<center><img src="https://assets.entrepreneur.com/content/3x2/2000/20200316195723-GettyImages-1136732916.jpeg?width=700&crop=2:1",height='300',width='600'></center>

##<h2> Introduction </h2>
The Springleaf Marketing Response challenge comprises of a high-dimensional dataset of anonymized customer information. Each row corresponds to one customer. The response variable is binary and labeled "target". Objective is to predict the target variable for every row in the test set.
The features have been anonymized and are comprised of a mix of continuous and categorical features. 

##<h2> Objective </h2>
To predict which customers will respond to a direct mail offer, using a large set of anonymized features.

**Data Pre-Processing**


Importing the libraries:

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [ ]:
#Changing working directory to Drive
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Project 4/')

Mounted at /content/drive


Reading the train & test dataset:

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Project 4/train.csv.zip')
df_test = pd.read_csv('/content/drive/MyDrive/Project 4/test.csv.zip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_train #Viewing the training dataset

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0008,VAR_0009,VAR_0010,VAR_0011,VAR_0012,VAR_0013,VAR_0014,VAR_0015,VAR_0016,VAR_0017,VAR_0018,VAR_0019,VAR_0020,VAR_0021,VAR_0022,VAR_0023,VAR_0024,VAR_0025,VAR_0026,VAR_0027,VAR_0028,VAR_0029,VAR_0030,VAR_0031,VAR_0032,VAR_0033,VAR_0034,VAR_0035,VAR_0036,VAR_0037,VAR_0038,VAR_0039,...,VAR_1896,VAR_1897,VAR_1898,VAR_1899,VAR_1900,VAR_1901,VAR_1902,VAR_1903,VAR_1904,VAR_1905,VAR_1906,VAR_1907,VAR_1908,VAR_1909,VAR_1910,VAR_1911,VAR_1912,VAR_1913,VAR_1914,VAR_1915,VAR_1916,VAR_1917,VAR_1918,VAR_1919,VAR_1920,VAR_1921,VAR_1922,VAR_1923,VAR_1924,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,997,997,997,147,198,173,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
1,4,H,7,53,4448,B,1.0,0.0,False,False,False,False,False,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,49,49,3,21,12,2,2,0,2,0,0,0,0,0,6546,6546,6125,76,1,1,9996,111,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
2,5,H,116,3,3464,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,997,997,45,46,46,1,0,0,3,0,97,0,97,997,999999997,999999997,999999997,999999997,1,997,9996,113,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
3,7,H,240,300,3200,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,120,30,996,996,15,188,106,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,RCC,0
4,8,R,72,261,2000,N,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30,1,17,17,76,443,230,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,BRANCH,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145226,290457,H,276,144,3000,C,2.0,3.0,False,False,False,False,False,2.0,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,...,998,998,998,998,9998,9998,9998,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,BRANCH,0
145227,290458,R,39,414,2821,C,3.0,1.0,False,False,False,False,False,3.0,1.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,400,400,997,997,49,389,139,3,2,0,5,1,1,1,1,0,30425,30425,30425,342,400,400,9996,39,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
145228,290459,R,103,31,2604,B,1.0,1.0,False,False,False,False,False,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,998,998,998,998,9998,9998,9998,4,0,0,0,8,4,8,4,0,999999996,999999996,999999996,999999996,400,400,55,76,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,BRANCH,0
145229,290461,H,78,12,4325,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145231 entries, 0 to 145230
Columns: 1934 entries, ID to target
dtypes: float64(477), int64(1406), object(51)
memory usage: 2.1+ GB


<u>Cleaning the dataset:</u>

*   The dataset is checked for any missing data and the respective columns are dropped.
*   Categorical features are converted to numeric
*   Filling in for the missing values

In [ ]:
#Detecting missing values from the dataset
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
VAR_0207,145231,1.000000
VAR_0213,145231,1.000000
VAR_0840,145231,1.000000
VAR_0214,145219,0.999917
VAR_0157,144311,0.993665
VAR_0158,143142,0.985616
VAR_0205,142974,0.984459
VAR_0206,142903,0.983970
VAR_0167,142664,0.982325
VAR_0177,141873,0.976878


In [ ]:
missing_data[missing_data["Total"] > 1] 

,Total,Percent
VAR_0207,145231,1.000000
VAR_0213,145231,1.000000
VAR_0840,145231,1.000000
VAR_0214,145219,0.999917
VAR_0157,144311,0.993665
...,...,...
VAR_0198,56,0.000386
VAR_0085,56,0.000386
VAR_0084,56,0.000386
VAR_0083,56,0.000386


In [ ]:
#Removing the columns with missing data
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_test = df_test.drop((missing_data[missing_data['Total'] > 1]).index,1)

In [ ]:
df_train #The no. of columns in the dataset has reduced

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0532,VAR_0533,VAR_0534,VAR_0535,VAR_0536,VAR_0537,VAR_0538,VAR_0539,VAR_0540,VAR_0541,VAR_0542,VAR_0543,VAR_0544,VAR_0545,VAR_0546,VAR_0547,VAR_0548,VAR_0549,VAR_0550,VAR_0551,VAR_0552,VAR_0553,VAR_0554,VAR_0555,VAR_0556,VAR_0557,VAR_0558,VAR_0559,VAR_0560,VAR_0561,VAR_0562,VAR_0563,VAR_0564,VAR_0565,...,VAR_1896,VAR_1897,VAR_1898,VAR_1899,VAR_1900,VAR_1901,VAR_1902,VAR_1903,VAR_1904,VAR_1905,VAR_1906,VAR_1907,VAR_1908,VAR_1909,VAR_1910,VAR_1911,VAR_1912,VAR_1913,VAR_1914,VAR_1915,VAR_1916,VAR_1917,VAR_1918,VAR_1919,VAR_1920,VAR_1921,VAR_1922,VAR_1923,VAR_1924,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,5,0,0,5,3,0,0,0,84,49463,0,116783,3230,1,998,998,998,998,66,9996,96,96,9996,9996,9996,9996,9996,96,96,0,0,0,0,0,...,1,997,997,997,147,198,173,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
1,4,H,7,53,4448,B,0,0,0,3,12,0,0,0,63,303472,0,346196,899,1,998,998,998,998,99,9996,96,96,9996,9996,9996,9996,9996,96,96,9,0,0,0,0,...,1,1,49,49,3,21,12,2,2,0,2,0,0,0,0,0,6546,6546,6125,76,1,1,9996,111,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
2,5,H,116,3,3464,C,0,0,0,1,2,0,0,0,48,94990,0,122601,0,1,998,998,998,998,87,9996,96,96,21,64,9996,9996,9996,96,96,25,0,0,0,0,...,1,1,997,997,45,46,46,1,0,0,3,0,97,0,97,997,999999997,999999997,999999997,999999997,1,997,9996,113,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
3,7,H,240,300,3200,C,0,0,0,1,6,2,0,0,65,20593,0,59490,286,1,998,998,400,400,77,9996,1,1,3,78,75,74,71,16,16,19,0,0,0,0,...,120,30,996,996,15,188,106,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,RCC,0
4,8,R,72,261,2000,N,0,0,0,1,8,0,0,0,84,10071,0,35708,947,1,1,30,998,998,57,9996,96,96,41,35,9996,9996,9996,96,96,0,0,0,0,0,...,30,1,17,17,76,443,230,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,BRANCH,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145226,290457,H,276,144,3000,C,0,0,0,2,3,1,0,0,68,124573,0,167505,2314,1,998,998,998,998,88,9996,1,1,3,82,81,70,9996,16,10,1,0,0,0,0,...,998,998,998,998,9998,9998,9998,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,BRANCH,0
145227,290458,R,39,414,2821,C,0,0,0,0,4,0,1,0,58,25768,0,62163,597,1,998,998,998,998,92,9996,1,1,3,73,61,60,59,25,24,9,0,0,0,0,...,400,400,997,997,49,389,139,3,2,0,5,1,1,1,1,0,30425,30425,30425,342,400,400,9996,39,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
145228,290459,R,103,31,2604,B,0,0,0,0,1,0,0,0,28,4744,0,7635,535,1,998,998,998,998,96,9996,96,1,5,2,9996,75,10,96,9,76,0,0,0,0,...,998,998,998,998,9998,9998,9998,4,0,0,0,8,4,8,4,0,999999996,999999996,999999996,999999996,400,400,55,76,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,BRANCH,0
145229,290461,H,78,12,4325,C,3,0,0,2,7,0,0,0,82,126971,0,196259,2239,1,998,998,998,998,79,9996,18,1,3,61,40,9990,38,18,18,42,0,0,0,0,...,30,1,0,0,5,258,182,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,1


In [ ]:
df_train["target"].unique()

array([0, 1])

In [ ]:
#Detecting columns with categorical features
df_train.select_dtypes(include=['O'])

,VAR_0001,VAR_0005,VAR_1934
0,H,C,IAPS
1,H,B,IAPS
2,H,C,IAPS
3,H,C,RCC
4,R,N,BRANCH
...,...,...,...
145226,H,C,BRANCH
145227,R,C,IAPS
145228,R,B,BRANCH
145229,H,C,IAPS


In [ ]:
df_train["VAR_0001"].unique()

array(['H', 'R', 'Q'], dtype=object)

In [ ]:
#Converting categorical features to numeric
df_train['0001H'] = df_train['VAR_0001'].map({'H': 1, 'R': 0, 'Q': 0})
df_train['0001R'] = df_train['VAR_0001'].map({'H': 0, 'R': 1, 'Q': 0})
df_train['0001Q'] = df_train['VAR_0001'].map({'H': 0, 'R': 0, 'Q': 1})
df_train.pop("VAR_0001")

df_test['0001H'] = df_test['VAR_0001'].map({'H': 1, 'R': 0, 'Q': 0})
df_test['0001R'] = df_test['VAR_0001'].map({'H': 0, 'R': 1, 'Q': 0})
df_test['0001Q'] = df_test['VAR_0001'].map({'H': 0, 'R': 0, 'Q': 1})
df_test.pop("VAR_0001")

0         R
1         R
2         R
3         R
4         H
         ..
145227    R
145228    R
145229    H
145230    H
145231    H
Name: VAR_0001, Length: 145232, dtype: object

In [ ]:
df_train["VAR_0005"].unique()

array(['C', 'B', 'N', 'S'], dtype=object)

In [ ]:
#Converting categorical features to numeric
df_train['0005C'] = df_train['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_train['0005B'] = df_train['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_train['0005N'] = df_train['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_train['0005S'] = df_train['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_train.pop("VAR_0005")

df_test['0005C'] = df_test['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_test['0005B'] = df_test['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_test['0005N'] = df_test['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_test['0005S'] = df_test['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_test.pop("VAR_0005")

0         B
1         C
2         C
3         B
4         C
         ..
145227    B
145228    C
145229    B
145230    B
145231    B
Name: VAR_0005, Length: 145232, dtype: object

In [ ]:
df_train["VAR_1934"].unique()

array(['IAPS', 'RCC', 'BRANCH', 'MOBILE', 'CSC'], dtype=object)

In [ ]:
#Converting categorical features to numeric
df_train['1934_IAPS'] = df_train['VAR_1934'].map({'IAPS': 1, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 0})
df_train['1934_RCC'] = df_train['VAR_1934'].map({'IAPS': 0, 'RCC': 1, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 0})
df_train['1934_BRANCH'] = df_train['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 1, 'MOBILE': 0, 'CSC': 0})
df_train['1934_MOBILE'] = df_train['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 1, 'CSC': 0})
df_train['1934_CSC'] = df_train['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 1})
df_train.pop("VAR_1934")

df_test['1934_IAPS'] = df_test['VAR_1934'].map({'IAPS': 1, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 0})
df_test['1934_RCC'] = df_test['VAR_1934'].map({'IAPS': 0, 'RCC': 1, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 0})
df_test['1934_BRANCH'] = df_test['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 1, 'MOBILE': 0, 'CSC': 0})
df_test['1934_MOBILE'] = df_test['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 1, 'CSC': 0})
df_test['1934_CSC'] = df_test['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 1})
df_test.pop("VAR_1934")

0           IAPS
1           IAPS
2           IAPS
3           IAPS
4           IAPS
           ...  
145227    BRANCH
145228      IAPS
145229    BRANCH
145230      IAPS
145231      IAPS
Name: VAR_1934, Length: 145232, dtype: object

In [ ]:
#Ensuring that all categorical columns are encoded
sum(df_train.select_dtypes(include=['O']))

0

In [ ]:
#Filling in the missing data
df_train = df_train.fillna(-1)
df_test = df_test.fillna(-1)

In [ ]:
df_train['target1'] = df_train["target"]
df_train.pop('target')
df_train

,ID,VAR_0002,VAR_0003,VAR_0004,VAR_0532,VAR_0533,VAR_0534,VAR_0535,VAR_0536,VAR_0537,VAR_0538,VAR_0539,VAR_0540,VAR_0541,VAR_0542,VAR_0543,VAR_0544,VAR_0545,VAR_0546,VAR_0547,VAR_0548,VAR_0549,VAR_0550,VAR_0551,VAR_0552,VAR_0553,VAR_0554,VAR_0555,VAR_0556,VAR_0557,VAR_0558,VAR_0559,VAR_0560,VAR_0561,VAR_0562,VAR_0563,VAR_0564,VAR_0565,VAR_0566,VAR_0567,...,VAR_1907,VAR_1908,VAR_1909,VAR_1910,VAR_1911,VAR_1912,VAR_1913,VAR_1914,VAR_1915,VAR_1916,VAR_1917,VAR_1918,VAR_1919,VAR_1920,VAR_1921,VAR_1922,VAR_1923,VAR_1924,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,0001H,0001R,0001Q,0005C,0005B,0005N,0005S,1934_IAPS,1934_RCC,1934_BRANCH,1934_MOBILE,1934_CSC,target1
0,2,224,0,4300,5,0,0,5,3,0,0,0,84,49463,0,116783,3230,1,998,998,998,998,66,9996,96,96,9996,9996,9996,9996,9996,96,96,0,0,0,0,0,0,0,...,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,1,0,0,1,1,1,1,1,0,0,0,0,0
1,4,7,53,4448,0,0,0,3,12,0,0,0,63,303472,0,346196,899,1,998,998,998,998,99,9996,96,96,9996,9996,9996,9996,9996,96,96,9,0,0,0,0,1,0,...,0,0,0,0,0,6546,6546,6125,76,1,1,9996,111,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,1,0,0,0,0,0,0,1,0,0,0,0,0
2,5,116,3,3464,0,0,0,1,2,0,0,0,48,94990,0,122601,0,1,998,998,998,998,87,9996,96,96,21,64,9996,9996,9996,96,96,25,0,0,0,0,0,0,...,0,97,0,97,997,999999997,999999997,999999997,999999997,1,997,9996,113,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,1,0,0,1,1,1,1,1,0,0,0,0,0
3,7,240,300,3200,0,0,0,1,6,2,0,0,65,20593,0,59490,286,1,998,998,400,400,77,9996,1,1,3,78,75,74,71,16,16,19,0,0,0,0,0,0,...,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,1,0,0,1,1,1,1,0,1,0,0,0,0
4,8,72,261,2000,0,0,0,1,8,0,0,0,84,10071,0,35708,947,1,1,30,998,998,57,9996,96,96,41,35,9996,9996,9996,96,96,0,0,0,0,0,0,0,...,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,0,1,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145226,290457,276,144,3000,0,0,0,2,3,1,0,0,68,124573,0,167505,2314,1,998,998,998,998,88,9996,1,1,3,82,81,70,9996,16,10,1,0,0,0,0,0,0,...,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,1,0,0,1,1,1,1,0,0,1,0,0,0
145227,290458,39,414,2821,0,0,0,0,4,0,1,0,58,25768,0,62163,597,1,998,998,998,998,92,9996,1,1,3,73,61,60,59,25,24,9,0,0,0,0,1,0,...,1,1,1,1,0,30425,30425,30425,342,400,400,9996,39,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,0,1,0,1,1,1,1,1,0,0,0,0,0
145228,290459,103,31,2604,0,0,0,0,1,0,0,0,28,4744,0,7635,535,1,998,998,998,998,96,9996,96,1,5,2,9996,75,10,96,9,76,0,0,0,0,0,0,...,8,4,8,4,0,999999996,999999996,999999996,999999996,400,400,55,76,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,0,1,0,0,0,0,0,0,0,1,0,0,0
145229,290461,78,12,4325,3,0,0,2,7,0,0,0,82,126971,0,196259,2239,1,998,998,998,998,79,9996,18,1,3,61,40,9990,38,18,18,42,0,0,0,0,0,0,...,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,1,0,0,1,1,1,1,1,0,0,0,0,1


**Training XGBoost Classifier**

In [ ]:
featureNames = df_train.columns[1:-1]

In [ ]:
#Taking a sub-sample of training set to reduce memory consumption
df_train = df_train.sample(n=20000)


#Training XGBoost classifier
dtrain = xgb.DMatrix(df_train[featureNames].values, label=df_train['target1'].values)

param = {'max_depth':2, 
         'eta':1, 
         'objective':'binary:logistic', 
         'eval_metric': 'auc'}
clf = xgb.train(param, dtrain, 20)


#Making predictions in batches
Prediction = df_test[['ID']]
Prediction['target1'] = np.nan
step = len(Prediction)/10000
for rows in range(0, len(Prediction), int(step)):
    Prediction.loc[rows:rows+step, "target"] = clf.predict(xgb.DMatrix(df_test.loc[rows:rows+step, featureNames].values))


#Saving the output file
#Prediction.to_csv("xgboost_output.csv", index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

Output of the model is saved in the working directory.